<a href="https://colab.research.google.com/github/jcgarcia18-create/CInema-Aguilas-Uas/blob/main/proyecto_final_sistemas_distribuidos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PROYECTO FINAL - SISTEMAS DISTRIBUIDOS
## Recomendador de libros con PySpark + TF-IDF
100 libros más descargados de Project Gutenberg

In [1]:
# 1. Instalamos dependencias (solo la primera vez)
!pip install pyspark beautifulsoup4 lxml -q

In [2]:
import re, os, requests
from bs4 import BeautifulSoup
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.ml.feature import Tokenizer, StopWordsRemover, CountVectorizer, IDF, Normalizer

# Stop any existing SparkSession to ensure a clean start
try:
    spark.stop()
except Exception:
    pass

spark = SparkSession.builder \
    .master("local[*]") \
    .appName("GutenbergRecommender") \
    .config("spark.sql.adaptive.enabled", "true") \
    .config("spark.driver.memory", "4g") \
    .getOrCreate()

sc = spark.sparkContext

In [3]:
# 3. Descarga automática de los 100 libros más populares
def descargar_top100():
    url = "https://www.gutenberg.org/browse/scores/top"
    headers = {'User-Agent': 'Mozilla/5.0'}
    soup = BeautifulSoup(requests.get(url, headers=headers).text, 'html.parser')

    libros = []
    for a in soup.find_all('a', href=True):
        href = a['href']
        if href.startswith('/ebooks/') and href[8:].isdigit():
            libros.append(int(href[8:]))

    top100 = libros[:100]
    os.makedirs("books", exist_ok=True)

    for eid in top100:
        txt_url = f"https://www.gutenberg.org/files/{eid}/{eid}-0.txt"
        r = requests.get(txt_url, headers=headers)
        if r.status_code == 200:
            with open(f"books/{eid}.txt", "wb") as f:
                f.write(r.content)
            print(f"Descargado {eid}")
        else:
            print(f"No se pudo descargar {eid} (status {r.status_code})")

descargar_top100()

Descargado 84
Descargado 2701
Descargado 1342
Descargado 1513
Descargado 46
Descargado 43
Descargado 25344
Descargado 8492
Descargado 11
Descargado 100
Descargado 145
Descargado 2641
Descargado 2542
Descargado 2554
Descargado 37106
Descargado 174
Descargado 16328
Descargado 1260
Descargado 1080
Descargado 844
Descargado 345
Descargado 64317
Descargado 67979
Descargado 76
Descargado 16389
Descargado 394
Descargado 1661
Descargado 4085
Descargado 6593
Descargado 2160
Descargado 1259
Descargado 98
Descargado 6761
Descargado 5197
Descargado 33944
Descargado 28054
Descargado 768
Descargado 3207
Descargado 1400
Descargado 5200
Descargado 205
Descargado 2591
Descargado 1184
Descargado 74
Descargado 55
Descargado 7370
No se pudo descargar 36034 (status 404)
Descargado 6130
Descargado 4300
No se pudo descargar 16119 (status 404)
Descargado 3206
Descargado 1998
Descargado 2600
Descargado 3296
No se pudo descargar 5740 (status 404)
Descargado 45
Descargado 77373
Descargado 1232
Descargado 23
Desc

In [4]:
# 4. Función de limpieza (elimina encabezado y pie de Gutenberg)
def limpiar_gutenberg(texto):
    inicio = texto.find("*** START OF THE PROJECT GUTENBERG EBOOK")
    if inicio == -1:
        inicio = texto.find("***START OF THE PROJECT GUTENBERG EBOOK")
    fin = texto.rfind("*** END OF THE PROJECT GUTENBERG EBOOK")
    if fin == -1:
        fin = texto.rfind("End of the Project Gutenberg EBook")

    if inicio != -1 and fin != -1:
        texto = texto[inicio:fin]

    # Normalización básica
    texto = re.sub(r'[^a-zA-Z\s]', ' ', texto.lower())
    texto = re.sub(r'\s+', ' ', texto).strip()
    return texto

In [5]:
# 5. Cargar y limpiar todos los libros
def cargar_libros():
    datos = []
    for archivo in os.listdir("books"):
        if archivo.endswith(".txt"):
            path = os.path.join("books", archivo)
            with open(path, "r", encoding="utf-8", errors="ignore") as f:
                texto = f.read()
            limpio = limpiar_gutenberg(texto)
            # Intentamos obtener título del nombre (opcional)
            titulo = archivo.replace(".txt", "")
            datos.append((archivo, titulo, limpio))
    return spark.createDataFrame(datos, ["id", "titulo", "texto"])

df = cargar_libros()
df.show(5, truncate=80)

+---------+------+--------------------------------------------------------------------------------+
|       id|titulo|                                                                           texto|
+---------+------+--------------------------------------------------------------------------------+
|16389.txt| 16389|start of the project gutenberg ebook illustration the enchanted april by eliz...|
|   46.txt|    46|start of the project gutenberg ebook a christmas carol a christmas carol in p...|
| 6761.txt|  6761|start of the project gutenberg ebook the adventures of ferdinand count fathom...|
|  514.txt|   514|start of the project gutenberg ebook little women little women by louisa may ...|
|27509.txt| 27509|start of the project gutenberg ebook the cia world factbook contents countrie...|
+---------+------+--------------------------------------------------------------------------------+
only showing top 5 rows



In [6]:
# 6. Preprocesamiento (tokenización + stop-words)
tokenizer = Tokenizer(inputCol="texto", outputCol="palabras")
df = tokenizer.transform(df)

remover = StopWordsRemover(inputCol="palabras", outputCol="filtradas")
df = remover.transform(df)

# Filtrar palabras muy cortas
filtrar_udf = udf(lambda palabras: [p for p in palabras if len(p) > 3], ArrayType(StringType()))
df = df.withColumn("filtradas", filtrar_udf("filtradas"))

In [7]:
# 7. CountVectorizer + TF-IDF
cv = CountVectorizer(inputCol="filtradas", outputCol="tf", vocabSize=20000, minDF=2)
cv_model = cv.fit(df)
df_tf = cv_model.transform(df)

idf = IDF(inputCol="tf", outputCol="tfidf")
idf_model = idf.fit(df_tf)
df_tfidf = idf_model.transform(df_tf)

# Normalización L2 (para similitud coseno)
normalizer = Normalizer(inputCol="tfidf", outputCol="norm", p=2.0)
df_final = normalizer.transform(df_tfidf)

df_final.cache()

DataFrame[id: string, titulo: string, texto: string, palabras: array<string>, filtradas: array<string>, tf: vector, tfidf: vector, norm: vector]

In [8]:
# 8. FUNCIÓN: Recomendar N libros similares
import numpy as np
from pyspark.sql.types import DoubleType

def recomendar(id_o_titulo, n=5):
    # Buscar el libro por id de archivo o por título
    query = df_final.filter((col("id") == id_o_titulo) | (col("titulo") == id_o_titulo))
    if query.count() == 0:
        print("Libro no encontrado")
        return

    fila_query = query.select("titulo", "norm").collect()[0]
    nombre = fila_query["titulo"]
    vector_query = fila_query["norm"].toArray()

    # Enviamos el vector de consulta a todos los workers
    bc_vec = sc.broadcast(vector_query)

    # UDF para calcular el producto punto con el vector de consulta
    def dot_with_query(v):
        if v is None:
            return 0.0
        return float(np.dot(v.toArray(), bc_vec.value))

    dot_udf = udf(dot_with_query, DoubleType())

    similitudes = df_final.withColumn("sim", dot_udf(col("norm")))

    resultado = similitudes.filter(col("titulo") != nombre) \
        .orderBy(col("sim").desc()) \
        .select("titulo", "sim") \
        .limit(n)

    print(f"Si te gusta: {nombre}")
    print("Te recomendamos:")
    for fila in resultado.collect():
        print(f"   → {fila.titulo} (sim: {fila.sim:.4f})")

    return resultado

In [11]:
# 9. FUNCIÓN: M palabras más características (versión corregida y con títulos)
def palabras_clave_por_titulo(titulo_busqueda, m=10):
    fila = df_final.filter(col("titulo") == titulo_busqueda).collect()
    if not fila:
        print("Título no encontrado")
        return

    fila = fila[0]
    vector = fila["tfidf"]
    vocab = cv_model.vocabulary

    # Convertir índices a enteros normales (corrección del error numpy.int32)
    pesos = [(float(vector[int(i)]), vocab[int(i)]) for i in vector.indices]

    # Ordenar por mayor peso TF-IDF
    top = sorted(pesos, reverse=True)[:m]

    print(f"Palabras clave de: {titulo_busqueda}")
    for peso, palabra in top:
        print(f"   {palabra}: {peso:.4f}")


In [13]:
# 10. EJEMPLOS REALES
print("RECOMENDACIONES")
recomendar("1342.txt", n=7)  # Pride and Prejudice (mantiene archivo)

print("\nPALABRAS CLAVE")
palabras_clave_por_titulo("84")  # Frankenstein


RECOMENDACIONES
Si te gusta: 1342
Te recomendamos:
   → 84 (sim: 0.1094)
   → 42324 (sim: 0.1085)
   → 41445 (sim: 0.1079)
   → 1260 (sim: 0.1053)
   → 768 (sim: 0.0637)
   → 4085 (sim: 0.0468)
   → 2160 (sim: 0.0400)

PALABRAS CLAVE
Palabras clave de: 84
   clerval: 185.6320
   justine: 142.2679
   elizabeth: 104.0890
   felix: 88.0005
   frankenstein: 84.9502
   safie: 78.6576
   geneva: 59.1202
   agatha: 51.3782
   ingolstadt: 50.3409
   cottagers: 45.2560
